In [1]:
import sys
import os
import numpy as np
from pathlib import Path
os.chdir(Path(os.getcwd()).resolve().parents[1])
import setup
from methods import grid_selection_amanda_dynamic
from sklearn.model_selection import ParameterGrid



def writeResults(datasetID, bestScore, bestParams, clfName):
    path = "results/stream/dynamic/gridsearch_amanda_dynamic_EVL-{}.txt".format(clfName)
    file = open(path,"a") 
    string = "{}: {} using {} \n".format(datasetID, bestScore, bestParams)
    file.write(string)
    file.close() 


def main():
    is_windows = sys.platform.startswith('win')
    sep = '\\'
    
    if is_windows == False:
        sep = '/'

    path = os.getcwd()+sep+'data'+sep
    
    #loading datasets
    datasets = [setup.loadCDT, setup.loadCHT, setup.load2CDT, setup.load2CHT, setup.load4CR, setup.load4CRE_V1, 
                setup.load4CRE_V2, setup.load5CVT, setup.loadCSurr, setup.load4CE1CF, setup.loadUG_2C_2D, setup.loadMG_2C_2D, 
                setup.loadFG_2C_2D, setup.loadUG_2C_3D, setup.loadUG_2C_5D, setup.loadGEARS_2C_2D, setup.loadCheckerBoard, 
                setup.loadElecData, setup.loadKeystroke, setup.loadNOAADataset]
    
    arrClfName = ['SGD', 'NB', 'RF', 'LP', 'KNN']
    for clfName in arrClfName:
        print("**************** BEGIN of {} results ****************".format(clfName))
        batches=1
        poolSize = 300
        isBatchMode = False
        #testing grid search
        for i in range(len(datasets)):
            finalScore = 0
            best_grid={}
            dataValues, dataLabels, description = datasets[i](path, sep)

            #Train-test split
            availableQty = int(0.05*len(dataLabels))
            availableLabels = dataLabels[:availableQty] 
            availableData = dataValues[:availableQty]

            # 70/30 train/test data
            initialLabeledData = int(0.7*len(availableLabels))
            sizeOfBatch = int((len(availableLabels)-initialLabeledData)/batches)

            print("{}: {} batches of {} instances".format(description, batches, sizeOfBatch))

            tuned_params = [{"sizeOfBatch":[sizeOfBatch], 
                             "batches":[batches], "poolSize":[poolSize], "isBatchMode":[isBatchMode], 
                             "initialLabeledData":[initialLabeledData], "clfName":[clfName]}]
            if clfName == 'LP' or clfName == 'KNN':
                tuned_params[0].update({"K":[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]})

            for g in ParameterGrid(tuned_params):
                averageAccuracy=0
                gs = grid_selection_amanda_dynamic.run(**g)

                try:
                    gs.fit(availableData, availableLabels)
                    averageAccuracy = np.mean(gs.predict())
                    print(averageAccuracy, g)
                    if finalScore < averageAccuracy:
                        finalScore = averageAccuracy
                        best_grid = g
                except Exception:
                    print("An error occured in ", description, g)
                    #raise Exception

            print(finalScore)
            print(best_grid)
            print("=======================================================================================================")

            writeResults(description, finalScore, best_grid, clfName)
        print("******** END of {} results ********".format(clfName))
    
if __name__ == "__main__":
    main()


**************** BEGIN of SGD results ****************
One Class Diagonal Translation. 2 Dimensional data.: 1 batches of 240 instances
100.0 {'batches': 1, 'clfName': 'SGD', 'initialLabeledData': 560, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 240}
100.0
{'batches': 1, 'clfName': 'SGD', 'initialLabeledData': 560, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 240}
One Class Horizontal Translation. 2 Dimensional data.: 1 batches of 240 instances
95.83 {'batches': 1, 'clfName': 'SGD', 'initialLabeledData': 560, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 240}
95.83
{'batches': 1, 'clfName': 'SGD', 'initialLabeledData': 560, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 240}
Two Classes Diagonal Translation. 2 Dimensional data: 1 batches of 240 instances
95.83 {'batches': 1, 'clfName': 'SGD', 'initialLabeledData': 560, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 240}
95.83
{'batches': 1, 'clfName': 'SGD', 'initialLabeledData': 560, 'isBatchMod

C:\Users\raul\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


One Class Surrounding another Class. Bidimensional.: 1 batches of 830 instances
An error occured in  One Class Surrounding another Class. Bidimensional. {'batches': 1, 'clfName': 'SGD', 'initialLabeledData': 1934, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 830}
0
{}
Four Classes Expanding and One Class Fixed. Bidimensional.: 1 batches of 2599 instances
An error occured in  Four Classes Expanding and One Class Fixed. Bidimensional. {'batches': 1, 'clfName': 'SGD', 'initialLabeledData': 6063, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 2599}
0
{}
Two Bidimensional Unimodal Gaussian Classes.: 1 batches of 1500 instances
An error occured in  Two Bidimensional Unimodal Gaussian Classes. {'batches': 1, 'clfName': 'SGD', 'initialLabeledData': 3500, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 1500}
0
{}
Two Bidimensional Multimodal Gaussian Classes.: 1 batches of 3000 instances
An error occured in  Two Bidimensional Multimodal Gaussian Classes. {'batches': 1, 'cl

An error occured in  Four Classes Expanding and One Class Fixed. Bidimensional. {'batches': 1, 'clfName': 'NB', 'initialLabeledData': 6063, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 2599}
0
{}
Two Bidimensional Unimodal Gaussian Classes.: 1 batches of 1500 instances
An error occured in  Two Bidimensional Unimodal Gaussian Classes. {'batches': 1, 'clfName': 'NB', 'initialLabeledData': 3500, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 1500}
0
{}
Two Bidimensional Multimodal Gaussian Classes.: 1 batches of 3000 instances
An error occured in  Two Bidimensional Multimodal Gaussian Classes. {'batches': 1, 'clfName': 'NB', 'initialLabeledData': 7000, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 3000}
0
{}
Two Bidimensional Classes as Four Gaussians.: 1 batches of 3000 instances
An error occured in  Two Bidimensional Classes as Four Gaussians. {'batches': 1, 'clfName': 'NB', 'initialLabeledData': 7000, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 3000}
0

C:\Users\raul\Desktop\GITHUB\Systems-Engineering\Dissertation\source\classifiers.py:131: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return RandomForestClassifier(n_estimators=num_trees, max_features=max_features).fit(X, y)


...

An error occured in  Four Classes Rotating with Expansion V1. Bidimensional. {'K': 2, 'batches': 1, 'clfName': 'KNN', 'initialLabeledData': 4375, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 1875}
An error occured in  Four Classes Rotating with Expansion V1. Bidimensional. {'K': 3, 'batches': 1, 'clfName': 'KNN', 'initialLabeledData': 4375, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 1875}
An error occured in  Four Classes Rotating with Expansion V1. Bidimensional. {'K': 4, 'batches': 1, 'clfName': 'KNN', 'initialLabeledData': 4375, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 1875}
An error occured in  Four Classes Rotating with Expansion V1. Bidimensional. {'K': 5, 'batches': 1, 'clfName': 'KNN', 'initialLabeledData': 4375, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 1875}
An error occured in  Four Classes Rotating with Expansion V1. Bidimensional. {'K': 6, 'batches': 1, 'clfName': 'KNN', 'initialLabeledData': 4375, 'isBatchMode': False, 'poolSiz

An error occured in  One Class Surrounding another Class. Bidimensional. {'K': 8, 'batches': 1, 'clfName': 'KNN', 'initialLabeledData': 1934, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 830}
An error occured in  One Class Surrounding another Class. Bidimensional. {'K': 9, 'batches': 1, 'clfName': 'KNN', 'initialLabeledData': 1934, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 830}
An error occured in  One Class Surrounding another Class. Bidimensional. {'K': 10, 'batches': 1, 'clfName': 'KNN', 'initialLabeledData': 1934, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 830}
An error occured in  One Class Surrounding another Class. Bidimensional. {'K': 11, 'batches': 1, 'clfName': 'KNN', 'initialLabeledData': 1934, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 830}
An error occured in  One Class Surrounding another Class. Bidimensional. {'K': 12, 'batches': 1, 'clfName': 'KNN', 'initialLabeledData': 1934, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch

An error occured in  Two Bidimensional Classes as Four Gaussians. {'K': 2, 'batches': 1, 'clfName': 'KNN', 'initialLabeledData': 7000, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 3000}
An error occured in  Two Bidimensional Classes as Four Gaussians. {'K': 3, 'batches': 1, 'clfName': 'KNN', 'initialLabeledData': 7000, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 3000}
An error occured in  Two Bidimensional Classes as Four Gaussians. {'K': 4, 'batches': 1, 'clfName': 'KNN', 'initialLabeledData': 7000, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 3000}
An error occured in  Two Bidimensional Classes as Four Gaussians. {'K': 5, 'batches': 1, 'clfName': 'KNN', 'initialLabeledData': 7000, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 3000}
An error occured in  Two Bidimensional Classes as Four Gaussians. {'K': 6, 'batches': 1, 'clfName': 'KNN', 'initialLabeledData': 7000, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 3000}
An error occured in  Two 

An error occured in  Two Rotating Gears (Two classes. Bidimensional). {'K': 8, 'batches': 1, 'clfName': 'KNN', 'initialLabeledData': 7000, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 3000}
An error occured in  Two Rotating Gears (Two classes. Bidimensional). {'K': 9, 'batches': 1, 'clfName': 'KNN', 'initialLabeledData': 7000, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 3000}
An error occured in  Two Rotating Gears (Two classes. Bidimensional). {'K': 10, 'batches': 1, 'clfName': 'KNN', 'initialLabeledData': 7000, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 3000}
An error occured in  Two Rotating Gears (Two classes. Bidimensional). {'K': 11, 'batches': 1, 'clfName': 'KNN', 'initialLabeledData': 7000, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 3000}
An error occured in  Two Rotating Gears (Two classes. Bidimensional). {'K': 12, 'batches': 1, 'clfName': 'KNN', 'initialLabeledData': 7000, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 3000}
0


83.88 {'K': 11, 'batches': 1, 'clfName': 'KNN', 'initialLabeledData': 634, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 273}
85.35 {'K': 12, 'batches': 1, 'clfName': 'KNN', 'initialLabeledData': 634, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 273}
85.35
{'K': 7, 'batches': 1, 'clfName': 'KNN', 'initialLabeledData': 634, 'isBatchMode': False, 'poolSize': 300, 'sizeOfBatch': 273}
******** END of KNN results ********
